# Multi-Layer Perceptron

In [1]:
from sklearn.metrics import r2_score, max_error, mean_absolute_error, mean_absolute_percentage_error
from dataset_storing import cal_fire_temps, k_fold, fire_X, fire_centroid_x, fire_centroid_y, fire_area
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from tensorflow.keras.metrics import R2Score, MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.losses import CosineSimilarity


this is working


2023-12-28 10:11:28.403827: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-28 10:11:28.477785: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-28 10:11:28.812728: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 10:11:28.814524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 10:11:28.887932: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Predicting area

In [2]:
fire_X.head(1)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,17.88,15.862,15.863,15.862,16.433,17.88,15.862,17.88,17.88,16.433,...,20.896,18.781,20.896,18.781,18.781,20.896,20.896,18.781,21.254,20.896


In [3]:
split_indices = k_fold.split(fire_X, fire_centroid_x)

accuracy_area_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    model = Sequential([ 
        Dense(256, activation='relu'),  
        Dense(128, activation='relu'),
        Dense(64, activation='sigmoid'),
        Dense(1, activation='linear'),   
    ]) 
    
    model.compile(loss='mse', optimizer='adam')
    
    train_X = fire_X.iloc[train].values.tolist()
    train_y = fire_area.iloc[train].values.tolist()
    test_X = fire_X.iloc[test].values.tolist()
    test_y = fire_area.iloc[test].values.reshape((len(test), 1))

    model.fit(train_X, train_y, epochs=10)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_area_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_area_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_area_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_area_dict["r2_score_test"].append(np.float128(r2_score(test_y, y_pred)))


Epoch 1/10
331/331 [==============================] - 2s 3ms/step - loss: 1676594842173440.0000
Epoch 2/10
331/331 [==============================] - 1s 3ms/step - loss: 1676594842173440.0000
Epoch 3/10
331/331 [==============================] - 1s 3ms/step - loss: 1676594976391168.0000
Epoch 4/10
331/331 [==============================] - 1s 3ms/step - loss: 1676593499996160.0000
Epoch 5/10
331/331 [==============================] - 1s 3ms/step - loss: 1676595110608896.0000
Epoch 6/10
331/331 [==============================] - 1s 3ms/step - loss: 1676594439520256.0000
Epoch 7/10
331/331 [==============================] - 1s 3ms/step - loss: 1676593634213888.0000
Epoch 8/10
331/331 [==============================] - 1s 3ms/step - loss: 1676593902649344.0000
Epoch 9/10
331/331 [==============================] - 1s 3ms/step - loss: 1676594305302528.0000
Epoch 10/10
331/331 [==============================] - 1s 3ms/step - loss: 1676593365778432.0000
Trial 0:
331/331 [=====================

In [4]:
accuracy_area_df = pd.DataFrame(accuracy_area_dict, index=range(1, 26))
accuracy_area_df.to_csv("data/mlp_area.csv")

X coord of the centroid

In [5]:
split_indices = k_fold.split(fire_X, fire_centroid_x)

accuracy_x_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    model = Sequential([ 
        Dense(256, activation='relu'),  
        Dense(128, activation='relu'),
        Dense(64, activation='sigmoid'),
        Dense(1, activation='linear'),   
    ]) 
    
    model.compile(loss='mse', optimizer='adam')
    
    train_X = fire_X.iloc[train].values.tolist()
    train_y = fire_centroid_x.iloc[train].values.tolist()
    test_X = fire_X.iloc[test].values.tolist()
    test_y = fire_centroid_x.iloc[test].values.reshape((len(test), 1))

    model.fit(train_X, train_y, epochs=10)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_x_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_x_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_x_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_x_dict["r2_score_test"].append(np.float128(r2_score(test_y, y_pred)))


Epoch 1/10
331/331 [==============================] - 2s 5ms/step - loss: 36368187392.0000
Epoch 2/10
331/331 [==============================] - 2s 5ms/step - loss: 36368166912.0000
Epoch 3/10
331/331 [==============================] - 2s 5ms/step - loss: 36368162816.0000
Epoch 4/10
331/331 [==============================] - 2s 5ms/step - loss: 36368146432.0000
Epoch 5/10
331/331 [==============================] - 2s 5ms/step - loss: 36368101376.0000
Epoch 6/10
331/331 [==============================] - 2s 5ms/step - loss: 36368089088.0000
Epoch 7/10
331/331 [==============================] - 2s 5ms/step - loss: 36368084992.0000
Epoch 8/10
331/331 [==============================] - 2s 5ms/step - loss: 36368044032.0000
Epoch 9/10
331/331 [==============================] - 2s 5ms/step - loss: 36368039936.0000
Epoch 10/10
331/331 [==============================] - 2s 5ms/step - loss: 36368027648.0000
Trial 0:
331/331 [==============================] - 1s 1ms/step
-0.0005264351090437103
Ep

In [6]:
accuracy_x_df = pd.DataFrame(accuracy_x_dict, index=range(1, 26))
accuracy_x_df.to_csv("data/mlp_cent_x.csv")

y coord of the centroid

In [7]:
split_indices = k_fold.split(fire_X, fire_centroid_x)

accuracy_y_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    model = Sequential([ 
        Dense(256, activation='relu'),  
        Dense(128, activation='relu'),
        Dense(64, activation='sigmoid'),
        Dense(1, activation='linear'),   
    ]) 
    
    model.compile(loss='mse', optimizer='adam')
    
    train_X = fire_X.iloc[train].values.tolist()
    train_y = fire_centroid_x.iloc[train].values.tolist()
    test_X = fire_X.iloc[test].values.tolist()
    test_y = fire_centroid_x.iloc[test].values.reshape((len(test), 1))

    model.fit(train_X, train_y, epochs=10)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_y_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_y_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_y_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_y_dict["r2_score_test"].append(np.float128(r2_score(test_y, y_pred)))


Epoch 1/10
331/331 [==============================] - 3s 5ms/step - loss: 36368195584.0000
Epoch 2/10
331/331 [==============================] - 2s 5ms/step - loss: 36368195584.0000
Epoch 3/10
331/331 [==============================] - 2s 5ms/step - loss: 36368158720.0000
Epoch 4/10
331/331 [==============================] - 2s 5ms/step - loss: 36368162816.0000
Epoch 5/10
331/331 [==============================] - 2s 5ms/step - loss: 36368150528.0000
Epoch 6/10
331/331 [==============================] - 2s 5ms/step - loss: 36368138240.0000
Epoch 7/10
331/331 [==============================] - 1s 4ms/step - loss: 36368121856.0000
Epoch 8/10
331/331 [==============================] - 2s 5ms/step - loss: 36368101376.0000
Epoch 9/10
331/331 [==============================] - 2s 5ms/step - loss: 36368097280.0000
Epoch 10/10
331/331 [==============================] - 2s 5ms/step - loss: 36368056320.0000
Trial 0:
331/331 [==============================] - 0s 849us/step
-0.0005247926403424419


In [8]:
accuracy_y_df = pd.DataFrame(accuracy_y_dict, index=range(1, 26))
accuracy_y_df.to_csv("data/mlp_cent_y.csv")